### Imports and data paths

In [ ]:
import pickle
import pandas as pd
import numpy as np
import pickle5 as pickle

In [ ]:
PATH_random_iid='./results/TestSuite-random_iid_06_15_2021.pkl'
PATH_uniform_iid='./results/TestSuite-uniform_iid_06_16_2021.pkl'
PATH_uniform_non_iid="./results/TestSuite-uniform_non_iid_06_17_2021.pkl"
PATH_random_non_iid="./results/TestSuite-random_non_iid_06_17_2021.pkl"

### Load the data

In [ ]:
def load_pickle(path):
    with open(path, "rb") as fh:
        data = pickle.load(fh)
    return data

data_uniform_iid = load_pickle(PATH_uniform_iid)
data_uniform_non_iid = load_pickle(PATH_uniform_non_iid)
data_random_non_iid = load_pickle(PATH_random_non_iid)
data_random_iid = load_pickle(PATH_random_iid)

### Helper functions for wrangling the data for the tables

In [ ]:
# Get the number of epochs it takes to converge and return a list of configuration + convergence epochs
def get_convergence(results, epsilon = 0.0001, config = "uniform_non_iid"):
    
    setup = []
    for idx, run in enumerate(results): # Iterate through runs

        nr_nodes = run['nr_nodes']
        run_epochs = []
        run_test_acc = []
        run_train_loss = []
        
        for i in range(nr_nodes): # Iterate through nodes in run
            
            node_dict = run[f'node_{i}']
            train_acc = node_dict['train_accuracies']
            train_loss = node_dict['train_losses']
            test_acc = node_dict['test_accuracies']
            learning_rate = node_dict['lr']
            
            for j in range(len(train_acc)-1): # Iterate through training accuracies
                                
                if abs(train_acc[j+1]-train_acc[j]) < epsilon:
                    run_epochs.append(j+1)
                    run_train_loss.append(train_loss[j])
                    run_test_acc.append(test_acc[j])
                    break
                    
                # If the run does not converge in the given number of epochs, we just take the accuracies of the last epoch     
                elif j == len(train_acc)-2:
                    run_epochs.append(j+2)
                    run_train_loss.append(train_loss[j+1])
                    run_test_acc.append(test_acc[j+1])
            break

        setup.append([run['add_privacy_list'], learning_rate, run['graph'], round(np.mean(run_epochs)), round(np.mean(run_train_loss),4), round(np.mean(run_test_acc),4), run['nr_nodes']])
        
    # Reformat list results into pandas dataframe
    setup_df = pd.DataFrame(setup, columns = ["Private", "Learning_rate", "Graph", "Mean_epochs", "Mean_train_loss", "Mean_test_accuracy", "Nr_nodes"])
    # Create the result table for the paper
    setup_df = setup_df.sort_values(by=["Nr_nodes", "Private"])
    setup_df["Config"] = config
    cols = list(setup_df.columns)
    cols = cols[-2:] + cols[:-2]  
    setup_df = setup_df[cols]
    setup_df = setup_df.replace(True, " Private")
    setup_df = setup_df.replace(False, " ")  
    setup_df["Config"] = setup_df["Config"] + setup_df["Private"]
    setup_df = setup_df.drop(columns = "Private")
    setup_df['Mean_epochs'] = setup_df['Mean_epochs'].apply(str)
    
    return setup_df

### Create and save the tables for each setup

In [ ]:
results_uniform_non_iid = get_convergence(data_uniform_non_iid, epsilon = 0.0001, config = "Non-IID Uniform")

In [ ]:
results_random_non_iid = get_convergence(data_random_non_iid, epsilon = 0.0001, config = "Non-IID Random")

In [ ]:
results_uniform_iid = get_convergence(data_uniform_iid, epsilon = 0.0001, config = "IID Uniform")

In [ ]:
results_random_iid = get_convergence(data_random_iid, epsilon = 0.0001, config = "IID Random")

In [ ]:
# Merge all results
results = results_uniform_iid.append(results_random_iid).append(results_uniform_non_iid).append(results_random_non_iid)
results = results.sort_values(by = ["Nr_nodes", "Config"])
results = results.replace("50", ">50")

# Split results by number of nodes
results_04 = results[results["Nr_nodes"] == 4].drop(columns="Nr_nodes").set_index(["Config","Learning_rate"])
results_16 = results[results["Nr_nodes"] == 16].drop(columns="Nr_nodes").set_index(["Config","Learning_rate"])
results_32 = results[results["Nr_nodes"] == 32].drop(columns="Nr_nodes").set_index(["Config","Learning_rate"])

# Save the tables as .tex
results_04.to_latex(buf='./results_tables/4_epochs_table.tex', sparsify=True)
results_16.to_latex(buf='./results_tables/16_epochs_table.tex', sparsify=True)
results_32.to_latex(buf='./results_tables/32_epochs_table.tex', sparsify=True)

results_04